In [38]:
import requests
import re
import pandas as pd
import time

# Update the product URL for the review pages
product_url = "#"  # Amazon review URL for the iPhone 13

# Define headers to simulate a browser request
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515.131 Safari/537.36",
    "Accept-Language": "en-US,en;q=0.9"
}

# Function to fetch reviews using regular expressions
def fetch_reviews(page_number):
    # Generate URL for the specific page number
    url = f"{product_url}/ref=cm_cr_getr_d_paging_btm_next_{page_number}?pageNumber={page_number}"
    response = requests.get(url, headers=headers)
    
    if response.status_code == 200:
        # Regular expressions for finding rating components
        rating_regex = re.compile(r'data-hook="review-star-rating".*?>(\d+\.\d) out of 5 stars<\/span>', re.S)
        text_regex = re.compile(r'data-hook="review-body".*?<span>(.*?)<\/span>', re.S)
        
        # Apply regex to the page content
        ratings = rating_regex.findall(response.text)
        texts = text_regex.findall(response.text)
        
        # Combine the data into a list of dictionaries
        reviews = []
        for i in range(min(len(ratings), len(texts))):
            if ratings[i].strip() == "5.0":  # Filter for stars ratings only
                review = {
                    "rating": ratings[i].strip(),
                    "text": texts[i].strip()
                }
                reviews.append(review)
                if len(reviews) >= 20:  # Stop if we've reached 20 reviews
                    break
        
        return reviews
    else:
        print(f"Failed to retrieve page {page_number}")
        return []

# Function to iterate through multiple pages and gather reviews
def gather_reviews(max_pages=5):
    all_reviews = []
    for page in range(1, max_pages + 1):
        print(f"Fetching page {page}...")
        reviews = fetch_reviews(page)
        if not reviews:
            break
        all_reviews.extend(reviews)
        if len(all_reviews) >= 20:  # Stop if we've reached 20 reviews
            all_reviews = all_reviews[:20]  # Limit to top 20 reviews
            break
        time.sleep(2)  # Avoid overloading Amazon's servers with requests
    
    return all_reviews

# Fetch reviews and save them to a DataFrame
reviews = gather_reviews(max_pages=5)
df = pd.DataFrame(reviews)

# Save to a CSV file
df.to_csv("your-csv-name.csv", index=False)
print("File is saved "+ your-csv-name.csv)


Fetching page 1...
Fetching page 2...
Fetching page 3...
Top 20 5-star reviews saved to amazon_iphone_13_5star_reviews.csv


In [ ]:
D